In [140]:
import pandas as pd
import numpy as np

In [141]:
df = pd.read_csv('/home/ANANT/ajadhav/CategorizeUserReviews/out.csv',sep='delimiter')

/home/ANANT/ajadhav/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [142]:
df.columns = ['Text']

df.Text = df.Text.str.replace('\d+', '')
df.Text = df.Text.map(lambda x: x.lstrip('\t').rstrip('aAbBcC'))

In [143]:
Categories = ['costly expensive high big','easy to use feasible adaptable','good quality best awesome better','cheap low economical affordable']

CategoryCol = ['Costly','Easy_to_Use','Good_Quality','Cheap']

In [144]:
df.head()

,Text
0,Very oily and creamy. Not at all what I expect...
1,This palette was a decent price and I was look...
2,The texture of this concealer pallet is fantas...
3,I really can't tell what exactly this thing is...
4,"It was a little smaller than I expected, but t..."


In [145]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)


def LoopCalc(Temp):
    vector1 = text_to_vector(Temp)
    Cat0 = text_to_vector(Categories[0])
    Cat1 = text_to_vector(Categories[1])
    Cat2 = text_to_vector(Categories[2])
    Cat3 = text_to_vector(Categories[3])

    cosine0 = get_cosine(vector1, Cat0)
    cosine1 = get_cosine(vector1, Cat1)
    cosine2 = get_cosine(vector1, Cat2)
    cosine3 = get_cosine(vector1, Cat3)
    return cosine0,cosine1,cosine2,cosine3



In [146]:
StackValues =  df.Text.apply(lambda x: LoopCalc(x))

In [147]:
CosineProbabilites = pd.DataFrame(np.column_stack(list(zip(*StackValues))), columns=CategoryCol)


In [148]:
ReviewWithCosine = pd.concat([df, CosineProbabilites], axis=1, sort=False)

In [149]:
ReviewWithCosine['Neutral'] = ReviewWithCosine.iloc[:,1:].sum(axis=1)<=0
ReviewWithCosine.Neutral = ReviewWithCosine.Neutral*1

In [150]:
ReviewWithCosine.head()

,Text,Costly,Easy_to_Use,Good_Quality,Cheap,Neutral
0,Very oily and creamy. Not at all what I expect...,0.0,0.200000,0.000000,0.000000,0
1,This palette was a decent price and I was look...,0.0,0.000000,0.000000,0.000000,1
2,The texture of this concealer pallet is fantas...,0.0,0.081349,0.027116,0.000000,0
3,I really can't tell what exactly this thing is...,0.0,0.000000,0.000000,0.000000,1
4,"It was a little smaller than I expected, but t...",0.0,0.080000,0.000000,0.044721,0


In [151]:
b = np.zeros_like(ReviewWithCosine.iloc[:,1:6])
b[np.arange(len(ReviewWithCosine.iloc[:,1:6])), ReviewWithCosine.iloc[:,1:6].values.argmax(1)] = 1

In [152]:
ReviewWithCosine.iloc[:,1:6] = b

In [153]:
ReviewWithCosine.head()

,Text,Costly,Easy_to_Use,Good_Quality,Cheap,Neutral
0,Very oily and creamy. Not at all what I expect...,0.0,1.0,0.0,0.0,0.0
1,This palette was a decent price and I was look...,0.0,0.0,0.0,0.0,1.0
2,The texture of this concealer pallet is fantas...,0.0,1.0,0.0,0.0,0.0
3,I really can't tell what exactly this thing is...,0.0,0.0,0.0,0.0,1.0
4,"It was a little smaller than I expected, but t...",0.0,1.0,0.0,0.0,0.0


In [154]:
ReviewWithCosine.to_csv('ReviewsWithCategory.csv',index=False)

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
transformer  = TfidfVectorizer(lowercase= True,stop_words=stop,max_features=500)
transformer.fit(ReviewWithCosine.Text)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=500, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=set([u'all', u'just', u"don't", u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'don', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u"should've", u"haven't", u'do', u'them', u'his', u'very', u"you've", u..., u'a', u'off', u'i', u'm', u'yours', u"you'll", u'so', u'y', u"she's", u'the', u'having', u'once']),
        strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [155]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,ReviewWithCosine.iloc[:,1:6],test_size = 0.25,
                                                   random_state = 42)


In [156]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score
confusion_matrix(y_test.values.argmax(axis=1),y_pred.argmax(axis=1))

In [ ]:
accuracy_score(y_test.values.argmax(axis=1),y_pred.argmax(axis=1))

In [ ]:
f1_score(y_test.values.argmax(axis=1),y_pred.argmax(axis=1),average='weighted')

In [136]:
#----------------------

In [138]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
#bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

{'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]}


In [139]:

rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 36.3min
Exception in thread QueueManagerThread:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/ANANT/ajadhav/.local/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py", line 741, in _queue_management_worker
    recursive_terminate(p)
  File "/home/ANANT/ajadhav/.local/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py", line 24, in recursive_terminate
    _recursive_terminate_without_psutil(process)
  File "/home/ANANT/ajadhav/.local/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py", line 49, in _recursive_terminate_without_psutil
    _recursive_terminate(pr

KeyboardInterrupt: 

In [ ]:
y_predGrid = rf.predict(X_test)
accuracy_score(y_test.values.argmax(axis=1),y_predGrid.argmax(axis=1))